# Reviewing Jupyter Widgets: Layout of our Data Dashboard


In [ ]:
# Import exception handling
import dashboard

### Goal of this Notebook

In this notebook we will be developing a `main.py` module which will actually implement the full data dashboard. To achieve this, we will take our previously developed widgets and use Jupyter widget's layout mechanisms to create a more polished and user-friendly interface.  

### Steps You Will Take In This Notebook
In this notebook you will:
1. Import our previously developed widgets
2. Experiment with style and layout of widgets
3. Create a layout for our data dashboard
4. Create a function to produce our data dashboard
5. Export the `main.py` module

Designate that we are exporting selected code to the `main` module.

In [4]:
#| default_exp main

## Importing our Previously Developed Widgets

Let's start by importing the widgets we created last time for controlling parameters and for displaying the `pandas` dataframe and the plot.

In [ ]:
#| export
# %answer key/dashboard/main.py 6

import ipywidgets as widgets
# import year_range, window_size, poly_order, selected_data_grid, update_selected_datagrid, plot_view, output_plot


We can even display them to make sure they still work as expected.

> **Issues?** If they don't work as expected, change the beginning of the import statement above to `from dashboard.widgets` to `from key.dashboard.widgets`

In [6]:
# Display dataframe
selected_data_grid

NameError: name 'selected_data_grid' is not defined

In [ ]:
# Show plot_view Output widget
plot_view

In [ ]:
# Check the year_range widget works
year_range

You should be able to move the year range slider and see both the dataframe of selected data and the plot respond.

Assuming this is the case, you successfully used nbdev to create a usable module for generating some visual elements of your dashboard. That's going to be a big deal as you can now build a new notebook to develop your dashboard and don't have to include the code for all those widget elements you've previously developed, you can just import them and move on!


## Changing Style and Layout of Widgets

### Changing Widget Styling

In the previous notebook, we noted the `style` attribute when we set up the widgets to have enough room to display their full text descriptions. The `style` attribute is used to change non-layout related styling options for widgets.

We previously used it to set the width of the widgets when we created them:
```python
year_range = widgets.IntRangeSlider(description = 'Range of Years',
                                    style={'description_width': 'initial'})
```
But what if we already created the `year_range` widget without the `style` attribute? We can still change the style of the widget after it has been created. We just call `style` attribute we want to change directly:
```python
year_range.style.description_width = 'initial'
```

While we only used one very utilitarian style attribute, but it is worth noting that you can use the style attribute to change your widgets in a variety of aesthetic ways: changing their colors, fonts, and more. A full list of which attributes are accessible for each widget is [available in the Jupyter widgets documentation](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Styling.html#current-supported-attributes).


### Changing Widget Layout

Jupyter widgets have a `layout` attribute exposing a number of CSS properties that impact how widgets are laid out. 

One thing we can do is change the width and height of a widget.  Execute the following cells to see the effects of changing the width and height on the associated widgets:

In [ ]:
# Redisplay the year_range widget to see the changes
year_range

In [ ]:
#| export
# Change the width of the year_range widget (makes year range selection easier to see)
year_range.layout.width = '500px'

In [ ]:
# Display the poly_order widget
poly_order

In [ ]:
#| export
# Change the width of the poly_order widget
poly_order.layout.width = '140px'

In [ ]:
# Display the datagrid containing the data
selected_data_grid

In [ ]:
#| export
# Change the width *and height* of the selected_data_grid widget
selected_data_grid.layout.width = '350px'
selected_data_grid.layout.height = '200px'

## Creating the layout of your Dashboard

### Container Widgets

We an use container widgets for any number of reasons. Later on in this tutorial, we will use the Tab widget to create multistep web applications. Below, we will nest the `selected_data_grid` widget inside the Accordion widget to give the user the option to hide it away and make it smaller. Let's see what it looks like on its own first.

In [ ]:
#| export
selected_data_accordion = widgets.Accordion(titles=('Selected Data',))

In [ ]:
selected_data_accordion

Not a lot going on here. Why? Because we haven't given the Accordion a "child" widget to hold just yet. Let's go ahead and do that.

The data type of the `children` trait expected by Container widgets is a tuple. If you haven't encountered tuples before, tuples are similar to lists but they are immutable, meaning that their contents can't be change once you've created them. The syntax for a tuple with multiple elements is `('one', 'two', 'three')`. Just like a list but with parentheses (there's an exception to this analogy which is mentioned shortly).

Okay, so let's try and add the `selected_data_grid` widget to the accordion. 

In [ ]:
%%exception

selected_data_accordion.children = (selected_data_grid)

That's a lot of text in one error message. What happened here? Our error is telling us that the `children` trait expected a tuple, and not an Output widget. Do you remember when I said the syntax of a tuple looks the same as a list but with parenthesis? Well, the problem is parenthesis are used in many other situations in Python, so how can the Python interpreter tell the difference between a single element tuple and just a variable inside of parenthesis for another reason.  Tuples with one element are written with a comma following the single element, this makes it clear to the Python interpreter that you are creating a tuple and not just using parenthesis for another reason.  

**EXERCISE** Fix the next cell so that you properly set the children attribute equal to a tuple with one element, the `selected_data_grid` widget.

In [ ]:
#| export
# %answer key/dashboard/main.py 29

It seems that by default, the data isn't showing. The accordion apprears to be closed. We can change this using the `selected_index` trait of the accordion. Let's see what it's set to now.

In [ ]:
print(selected_data_accordion.selected_index)

If your accordion is closed, `selected_index` will be `None` and so this will print `None`. 
We can open the accordion programmatically by setting the `selected_index` to the index of the accordion we want to open. In this case there is only one. 

**EXERCISE**: Go ahead and try to open the accordion programmatically using the next cell.

In [ ]:
#| export
# %answer key/dashboard/main.py 33

We like this trick, because our users are probably more interested in our plot than the raw data anyways, so this keeps the raw data from being in the way.

### Add Descriptive Text with Links

Before we put everything together, let's add some widgets that provide information about the dashboard. Because we want to take advantage of hyperlinking (to properly cite our sources), we will use the `HTML` widget. The `HTML` widget allows you to display text that can be styled using HTML and CSS.  First let's export the HTML we will want to display as two constants.

In [ ]:
#| export
INTRO_TEXT = '''
<p><b>Curve Smoothing</b>
This tool is for smoothing and selecting global mean surface temperature data for visualization. Start by selecting a date
range, and then select the smoothing parameters you want to use. Then click through to the next step, where you will change properties
of the curve smoothing algorithm you selected and visualize the data.
</p>
'''
SOURCES_TEXT = '''
<p>
<b>About Global Mean Surface Temperature Data</b>
<a href="https://climate.nasa.gov/vital-signs/global-temperature/"
target="_blank">Global Temperature (NASA)</a>,
<a href="https://data.giss.nasa.gov/gistemp/graphs/"
target="_blank">GISS Surface Temperature Analysis (NASA)</a>
</p><p>
This site is based on data downloaded from the following site on 2024-06-17:
<a href="https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.csv"
target="_blank">Global Annual Mean Surface Air Temperature Change (NASA)</a>
'''

### Layout widget objects

In most cases, we can use the `Layout` object to set the layout of a particular widget on initialization.  Let's display the HTML variables we just defined in `HTML` widgets using the `Layout` object to set the width of the widgets to be the same.

**PRO TIP**: `Layout` is just a widget, so you can observe its traits and react to changes in those traits like any other widget.

In [ ]:
#| export
html_layout = widgets.Layout(max_width = '500px')
intro_text = widgets.HTML(value = INTRO_TEXT, layout = html_layout)
data_source_text = widgets.HTML(value = SOURCES_TEXT, layout = html_layout)

In [ ]:
intro_text

In [ ]:
data_source_text

### Arranging Widgets with HBox and VBox Containers

The arrangement where every widget is stacked one on top of the other isn't ideal for a data dashboard that we expect users to access from a desktop. Jupyter widgets has several container widgets to arrange widgets in various ways. Perhaps two of the most handy are the `HBox` and `VBox` widgets, which arrange widgets horizontally and vertically, respectively. If you are familiar with `FlexBox`, [those properties are available under the hood](https://ipywidgets.readthedocs.io/en/7.6.3/examples/Widget%20Styling.html#The-Flexbox-layout), but we will not cover them here (we will use one of those properties below). 

Let's create a `HBox` to put the curve parameter widgets (`window_size` and `poly_order`) side by side.

In [ ]:
#| export
# Define the widget containing the curve smoothing parameters
curve_parameters_layout = widgets.Layout(width='500px', justify_content='space-between')
curve_parameter_widgets = widgets.HBox(children=(window_size, poly_order), 
                                       layout=curve_parameters_layout)

In [ ]:
# Display the curve parameter widgets in a single container
curve_parameter_widgets

We used the `justify_content` (Flexbox) paramter in the layout to make it so space was inserted between the two children so that the total width they occupy is 500 pixels. 

Let's compare the width of the curve parameter controls to the width of the `years_range` slider.

In [ ]:
year_range

Nice! This looks good because this `HBox` is about as wide as our `year_range` slider.  Notice that we can pass in the children as a parameter to the widget, or change the children trait after the widget has already been instantiated. 

**EXERCISE**: Create a `VBox` vertical container that holds the descriptive text (`intro_text`, `data_source_text`) and `curve_parameter_widgets` and `year_range` widgets.  This will allow us to stack the widgets on top of each other.

In [ ]:
#| export
# %answer key/dashboard/main.py 47

# Create a VBox to hold the description and control widgets
# add children intro_text, data_source_text, year_range, curve_parameter_widgets


In [ ]:
# Display the left_vbox widget we just created
desc_and_ctrl_box

This look okay, I guess, but we could really use some more padding in between the widgets.

In [ ]:
desc_and_ctrl_box.layout.margin = '15px 0 15px 0' # top, right, bottom, left

**EXERCISE**: Hmmm... that seemed to add a little padding, but only to the outside of the `desc_and_ctrl_box` container. Can you guess how we might loop through and add padding to each of the child widgets? Try it below.

In [ ]:
#| export
# %answer key/dashboard/main.py 52

# how might we add padding to each of the widgets


That looks a lot better! Now take care of the right side of our dashboard by creating a `VBox` container containing the `selected_data_accordion` and `plot_view` widgets.

In [ ]:
#| export
# %answer key/dashboard/main.py 54

# Add a vertical box holding both table and plot visualizations of selected data


In [ ]:
data_box

Okay! Lets put the left and the right-hand boxes side by side to create the final form of our dashboard!

In [ ]:
#| export
main_widget = widgets.HBox(children = (desc_and_ctrl_box, data_box))

In [ ]:
main_widget

Depending on the width of your display, this might appear awfully squished together! Let's change a few more layout settings to clean things up.

In [ ]:
#| export
data_box.layout.margin = '0 0 0 30px'  # top, right, bottom, left
data_box.layout.align_items = 'flex-end'
selected_data_accordion.layout.width = '88%'
desc_and_ctrl_box.layout.min_width = '500px'

Wow, that looks pretty good!  There are several other ways to approach layout of widgets in Jupyter widgets, but this is a good start.  You can find more information in the [Jupyter Layout documentation](https://ipywidgets.readthedocs.io/en/stable/examples/Layout%20Templates.html).


It's time to export the `main` module for our dashboard. Remember that our `main_widget` contains all the children widgets, so we should be able to simply import `main` in another notebook to see the final product.

In [ ]:
from nbdev.export import nb_export

nb_export('02c_layout.ipynb', 'dashboard')

# Challenges with this Approach

Notice that even for a relatively simple dashboard, we had to spend quite a bit of time writing:
- multiple handler functions for the widgets since multiple controls were used,
- the handler functions needed to handle the validation of inputs,
- custom layout of widgets was necessary to have all the controls appear together.

As a web app's controls and logic get more complex, the complexity of the code to handle the controls and layout can grow burdensome.  This is why we will be exploring a more advanced approach in the next section of this tutorial.